# 📝 Handwritten Character Regconition using Machine Learning

This notebook will go through a machine learning project with the goal of classifying different handwritten characters including letters and digits.

To do this, we will be using data downloaded from [EMNIST (Extended MNIST)](https://www.kaggle.com/datasets/crawford/emnist). It consists of six different splits with details provided below: 
* EMNIST ByClass: 814,255 characters. 62 unbalanced classes.
* EMNIST ByMerge: 814,255 characters. 47 unbalanced classes.
* EMNIST Balanced:  131,600 characters. 47 balanced classes.
* EMNIST Letters: 145,600 characters. 26 balanced classes.
* EMNIST Digits: 280,000 characters. 10 balanced classes.
* EMNIST MNIST: 70,000 characters. 10 balanced classes.

Visual breakdown of EMNIST datasets is specified as follow:
<img src="../report/images/emnistdataset.jpg" width="400" height="800">

Please refer to the [EMNIST: an extension of MNIST to handwritten letters](https://arxiv.org/abs/1702.05373v1) paper for more details of the dataset. 

In this project, the `EMNIST Balanced` dataset will be used to reduce mis-classification errors due to capital and lower case letters and also has an equal number of samples per class. The `EMNIST Balanced` includes two main datasets: 
* **emnist-balanced-train.csv** - 112,800 images 
* **emnist-balanced-test.csv** - 18,800 images

Each image is stored in `CSV` files using a separated row and 785 columns (first column represents the class lable )


In [175]:
# Regular EDA (exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import sys  
sys.path.insert(0,'CNN')

from cnn import CNN
from layers import Conv2D, MaxPooling2D
from layers import Dense, Dropout, Flatten

In [176]:
train_data = pd.read_csv('../../data/emnist-balanced-train.csv')
test_data = pd.read_csv('../../data/emnist-balanced-test.csv')
mapp = pd.read_csv("../../data/emnist-balanced-mapping.txt", 
                   delimiter = ' ', index_col=0, header=None, squeeze=True)

In [177]:
train_data.head()

,45,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.524,0.525,0.526,0.527,0.528,0.529,0.530,0.531,0.532,0.533
0,36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,43,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
test_data.head()

,41,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.523,0.524,0.525,0.526,0.527,0.528,0.529,0.530,0.531,0.532
0,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [179]:
print(f'Train: {train_data.shape}, Test: {test_data.shape}, Map: {mapp.shape}')

Train: (112799, 785), Test: (18799, 785), Map: (47,)


In [180]:
# Define width and height of images
WIDTH = 28
HEIGHT = 28

In [181]:
# Split X and y
X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]

X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]

In [182]:
print(f'Train data: {X_train.shape}')
print(f'Train labels: {y_train.shape}')
print(f'Test data: {X_test.shape}')
print(f'Train labels: {y_test.shape}')

Train data: (112799, 784)
Train labels: (112799,)
Test data: (18799, 784)
Train labels: (18799,)


In [183]:
def rotate(image):
    image = image.reshape([HEIGHT, WIDTH])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

In [184]:
# Flip and rotate image
X_train = np.asarray(X_train)
X_train = np.apply_along_axis(rotate, 1, X_train)
print(f'Train data: {X_train.shape}')

X_test = np.asarray(X_test)
X_test = np.apply_along_axis(rotate, 1, X_test)
print(f'Test data: {X_test.shape}')

Train data: (112799, 28, 28)
Test data: (18799, 28, 28)


In [185]:
# Normalize data
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [186]:
# Reshaping for input to CNN
X_train = X_train.reshape([-1,HEIGHT,WIDTH,1])
X_test = X_test.reshape([-1,HEIGHT,WIDTH,1])

In [187]:
# number of classes
num_classes = y_train.nunique()

In [188]:
m = CNN()

m.add(Conv2D(filters=128, kernel_size=(5,5), padding = 'same', activation='relu',\
                 input_shape=(HEIGHT, WIDTH,1)))
m.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
m.add(Conv2D(filters=64, kernel_size=(3,3) , padding = 'same', activation='relu'))
m.add(MaxPooling2D(pool_size=(2,2)))

m.add(Flatten())
m.add(Dense(units=128, activation='relu'))
m.add(Dropout(.5))
m.add(Dense(units=num_classes, activation='softmax'))

m.summary()

                  Input Shape   Output Shape Activation  Parameters
Layer Name                                                         
Conv2D_1          (28, 28, 1)  (28, 28, 128)       relu        3328
MaxPooling2D_2  (28, 28, 128)  (14, 14, 128)       None           0
Conv2D_3        (14, 14, 128)   (14, 14, 64)       relu           1
MaxPooling2D_4   (14, 14, 64)     (7, 7, 64)       None           0
Flatten_5          (7, 7, 64)           3136       None           0
Dense_6                  3136            128       relu      401536
Dropout_7                 128            128       None           0
Dense_8                   128             47    softmax        6063
Total Parameters: 410928


In [189]:
m.compile_model(loss='cse', opt='adam', metrics=['accuracy'])

AttributeError: 'Optimizer' object has no attribute 'sgd'